In [ ]:
import xgboost as xgb

In [ ]:
import numpy as np
import pandas as pd
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
%matplotlib inline
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from DSI_Capstone_Steemit.utils.utils import(
    load_data_and_description,
)
from sklearn.model_selection import train_test_split

data,feature_names,data_desc = load_data_and_description(data_type='posts_tfidf')
data_desc['log total_payout_value'] = np.log(data_desc['total_payout_value'])



In [ ]:
idx1 = data_desc['log total_payout_value'] < 1.2
idx2 = data_desc['log total_payout_value'] >2.5

idx_not = (~idx1) & (~idx2)

data_desc = data_desc[~idx_not]
data = data[~idx_not.values,:]
y = data_desc['log total_payout_value'] >2.5

In [ ]:
value_counts = data_desc['category'].value_counts()

top_categories = value_counts.index[value_counts > np.percentile(data_desc['category'].value_counts(),97)]

idx = data_desc['category'].isin(top_categories)

data_desc['top category'] = idx.astype(int)

data_desc['top category listed'] = data_desc.ix[data_desc['top category'].values.astype(bool) ,'category']

data_desc['top category listed'] = data_desc['top category listed'].fillna('Other')

In [ ]:
import scipy.sparse as ssp


train_features = data_desc.ix[:,['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts',
                                'top category listed']]


train = pd.get_dummies(train_features)

num_image_urls = train['number of image urls'].values[:,0]
train.drop('number of image urls',axis = 1, inplace=True)

train['number of image urls'] = num_image_urls

training_names = train.columns

train_sparse = ssp.csr_matrix(train)
new_data = ssp.hstack([data,train_sparse])
train = new_data.tocsr()

In [ ]:
training_names

In [ ]:
all_features = list(feature_names) + list(training_names)

In [ ]:
from sklearn.utils import shuffle

In [ ]:

number_of_samples = 5000

# All samples
number_of_samples = train.shape[0]


index = shuffle(range(train.shape[0]))[0:number_of_samples]

X_train, X_test, y_train, y_test = train_test_split(
    train[index], y[index], test_size=0.33, random_state=42)



In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.grid_search import GridSearchCV



In [ ]:
# import xgboost as xgb
# from xgboost import XGBClassifier
# from sklearn.grid_search import GridSearchCV

# y1_list = []
# y2_list = []
# lr_list = [0.001,0.005,0.01,0.05,0.1,1]
# for lr in lr_list:
#     eval_set = [(X_train, y_train), (X_test, y_test)]
#     eval_metric = 'error'
#     model = XGBClassifier(learning_rate=lr, n_estimators=1000)
#     model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=False)

#     results = model.evals_result()
    
#     y1 = results['validation_0'][eval_metric]
#     y2 = results['validation_1'][eval_metric]

#     y1_list.append(y1)
#     y2_list.append(y2)    



In [ ]:
# epochs = len(results['validation_0'][eval_metric])
# x_axis = range(0, epochs)
# # plot log loss
# fig, ax = plt.subplots()

# colors = sns.color_palette()
# for i,y1 in enumerate(y1_list):
#     lr = lr_list[i]
#     ax.plot(x_axis, y1, label='Train:' + str(lr), color = colors[i])
#     ax.plot(x_axis, y2_list[i], label='Test:' + str(lr),color = colors[i])
# ax.legend(loc = (1,0.5))
# plt.ylabel('Error')
# plt.xlabel('Epoch')
# plt.title('XGBoost Error')
# plt.show()



In [ ]:
# xgb.plot_importance(model)

## Try and determine eta (learning rate) and number of trees

In [ ]:
print (2 + 0.0) / 100, (10 + 0.0) / 100

In [ ]:
cv_params = {'max_depth': [1,3,5,7], 
             'min_child_weight': [1,3,7],
             'colsample_bytree': [0.5,0.8,1],
             'reg_alpha':[0.01,0.1,0.2],
             'reg_lambda':[0.01,0.1,0.2]}


# cv_params = {'max_depth': [1], 
#              'min_child_weight': [3],
#              'colsample_bytree': [0.5]}



'''
lambda [default=1]
L2 regularization term on weights (analogous to Ridge regression)
This used to handle the regularization part of XGBoost. Though many data scientists don’t use it often, it should be explored to reduce overfitting.
alpha [default=0]
L1 regularization term on weight (analogous to Lasso regression)
Can be used in case of very high dimensionality so that the algorithm runs faster when implemented
'''
ind_params = {'learning_rate': 0.05, 'n_estimators': 100, 'seed':0, 
             'objective': 'binary:logistic'}
model = xgb.XGBClassifier(**ind_params)
optimized_GBM = GridSearchCV(model, 
                            cv_params, 
                             scoring ='accuracy', 
                             cv = 5, 
                             n_jobs = -1, 
                             verbose = False) 

In [ ]:
optimized_GBM.fit(X_train, y_train.values)


In [ ]:
gridsearch_crossval_results = joblib.load('grid_results/gridsearch_crossval_results')
gridsearch_model = joblib.load('grid_results/gridsearch_model')

In [ ]:
xgb_best_model = gridsearch_model.best_estimator_

In [ ]:
y = data_desc['log total_payout_value'] >2.5

data_desc['true label'] = y

desc_split_train,desc_split_test = train_test_split(
    data_desc, test_size=0.33, random_state=42)





X_train, X_test, y_train, y_test = train_test_split(
    train, y, test_size=0.33, random_state=42)

xgb_best_params = gridsearch_model.best_params_

best_estimator = xgb.XGBClassifier(n_estimators=500, 
                                   **xgb_best_params)

In [ ]:
# from sklearn.model_selection import KFold

# kf = KFold(n_splits=5, shuffle=True)
# kf.get_n_splits(train)

# print(kf)  
# probabilities_list = []
# train_index_list = []
# for train_index, test_index in kf.split(train):
#     train_index_list.append(train_index)
#     X_train, X_test = train[train_index], train[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     best_estimator.fit(X_train,y_train)
    
#     probabilities = best_estimator.predict_proba(X_test,y_test)
#     probabilities_list.append(probabilities)

In [ ]:
best_estimator.fit(X_train,y_train)

In [ ]:
y_score = best_estimator.score(X_test,y_test) # Predict using our testdmat
predictions = best_estimator.predict(X_test) # Predict using our testdmat
probabilities = best_estimator.predict_proba(X_test)
print y_score

In [ ]:
all_features[-1]

In [ ]:
sort_order = np.argsort(best_estimator.feature_importances_)[::-1]
features_ordered = np.array(all_features)[sort_order]
values_ordered = best_estimator.feature_importances_[sort_order]

In [ ]:
top = 50
fig,ax = plt.subplots(1,1,figsize = (10,10))
sns.barplot(x = values_ordered[0:top],y = features_ordered[0:top], ax = ax)

In [ ]:
print y_score,  y_test.mean()

In [ ]:
desc_split_test['predictions'] = predictions
desc_split_test['probability of 1'] = probabilities[:,1]



In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize = (10,5))

sns.distplot(np.log(desc_split_test.ix[:,'total_payout_value']),ax =ax1, 
                 kde = False)

# ax1.axvline(np.log(percentile_25))
# ax1.axvline(np.log(percentile_50))
# ax1.axvline(np.log(percentile_75))

# ax1.text(x = np.log(percentile_25), y = 2000,s='25%',ha = 'center')
# ax1.text(x = np.log(percentile_50), y = 2000,s='50%',ha = 'center')
# ax1.text(x = np.log(percentile_75), y = 2000,s='60%',ha = 'center')
ax1.set_title('Prediction Label Split',y = 1.1)
ax1.set_xlabel('log(total_payout_value)')


for label in np.sort(desc_split_test['predictions'].unique()):
    idx = desc_split_test['predictions'] == label
    sns.distplot(np.log(desc_split_test.ix[idx,'total_payout_value']),ax = ax2,
                 kde = False, label = str(label))

ax2.legend()
ax2.set_title('Predictions of Gradient Boosting Model')
ax2.set_xlabel('log(total_payout_value)')
fig.tight_layout()
plt.savefig('Distribution Plot - New.png')

In [ ]:
# desc_split_test['true label'] = (desc_split_test['total_payout_value'] > np.median(data_desc['total_payout_value'])).astype(int)
# desc_split_test_short = desc_split_test.ix[:,['predictions','probability of 1','total_payout_value','true label']]
from sklearn.metrics import confusion_matrix
cm =  confusion_matrix(y_pred= desc_split_test['predictions'], y_true=desc_split_test['true label'])

In [ ]:
cm_normal = (cm + 0.0)/desc_split_test.shape[0]

In [ ]:
sns.heatmap(cm,cmap='Blues', annot=True,xticklabels=[1,2,3,4],yticklabels=[1,2,3,4],
            fmt='d')
plt.savefig('Confusion Matrix.png')

In [ ]:
sns.heatmap(cm_normal,cmap='Blues', annot=True,xticklabels=[1,2,3,4],yticklabels=[1,2,3,4],
            fmt='f')
plt.tight_layout()
# plt.savefig('Confusion Matrix.png')

In [ ]:
desc_split_test['Difference'] = abs(desc_split_test['predictions'] - desc_split_test['true label'])

In [ ]:
desc_split_test['length of article'] = desc_split_test['body'].fillna('').str.split(' ').apply(len)

In [ ]:
sns.distplot(np.log(desc_split_test['length of article']))
plt.axvline(np.log(desc_split_test['length of article'].mean()))

In [ ]:
desc_split_test.columns

In [ ]:
desc_split_test['author_reputation_scaled'].max()

In [ ]:
desc_split_test['Log Difference'] = np.log(desc_split_test['length of article'].values )
desc_split_test.ix[desc_split_test['Difference'] > 2,['Log Difference','true label',
                                                      'predictions',
                                                      'Difference',
                                                      'length of article',
                                                      'number of body mentions',
                                                      'number of image urls',
                                                     'whale', u'body whale mentions',
                                                     'author_reputation_scaled']].sort_values('predictions')

In [ ]:
sns.regplot(x = 'length of article',y = 'total_payout_value',data = desc_split_test)